<a href="https://colab.research.google.com/github/BigyanBhatta/CNNs/blob/main/Pizza_or_not(Transfer_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d carlosrunner/pizza-not-pizza

 94% 95.0M/101M [00:04<00:00, 23.6MB/s]
100% 101M/101M [00:04<00:00, 24.4MB/s] 


In [ ]:
import zipfile
zip_path = zipfile.ZipFile('/content/pizza-not-pizza.zip', 'r')
zip_path.extractall('/content')
zip_path.close()


In [ ]:
import keras
import tensorflow as tf
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base= VGG16(include_top = False,
              weights = 'imagenet',
              input_shape=(150, 150, 3))


58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 16812353 (64.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
conv_base.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 2097665 (8.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
# generator- convert data into batches

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/pizza_not_pizza',
    labels = 'inferred',
    batch_size = 32,
    label_mode = 'int',
    image_size = (150, 150)
)

Found 1966 files belonging to 2 classes.


In [ ]:
# normalize

def process(image, label):
  image = tf.cast(image/ 255., tf.float32)
  return image, label

train_ds = train_ds.map(process)

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs = 10)

Epoch 1/10
62/62 [==============================] - 14s 96ms/step - loss: 0.4174 - accuracy: 0.8189
Epoch 2/10
62/62 [==============================] - 4s 65ms/step - loss: 0.2046 - accuracy: 0.9212
Epoch 3/10
62/62 [==============================] - 5s 65ms/step - loss: 0.1235 - accuracy: 0.9568
Epoch 4/10
62/62 [==============================] - 4s 65ms/step - loss: 0.0667 - accuracy: 0.9852
Epoch 5/10
62/62 [==============================] - 5s 73ms/step - loss: 0.0371 - accuracy: 0.9944
Epoch 6/10
62/62 [==============================] - 4s 66ms/step - loss: 0.0192 - accuracy: 0.9990
Epoch 7/10
62/62 [==============================] - 5s 70ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 8/10
62/62 [==============================] - 4s 67ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 9/10
62/62 [==============================] - 5s 67ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 10/10
62/62 [==============================] - 5s 76ms/step - loss: 0.0033 - accuracy: 1.0000
